# Substitute Days Covered

In this Jupyter notebook, the Data Analyst Teacher Absent Report that has been created in Frontline is used. The substitutes are selected for teachers and sgis that called out and duplicate days in which they might have covered multiple teachers are dropped. A value of one is given to each day in which the substitute covered. They are separated by the two sites (Delano and Lost Hills) then summed up for each substitute. A final column is created with total days that they have covered.

A second portion of the code looks to find active (days worked > 0) and inactive (days worked = 0) substitutes. It will be done by combining the dataframe from the first part with the list of substitutes. Then the dataframe will be separated into two dataframes based on the criteria above.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load Substitute list
substitutes_roster = pd.read_csv(r"C:\Users\derek.castleman\Desktop\2025_02_25_20265_Castleman Substitute List_664664159_.csv")

# Load Data Analyst Teacher Absent Report
substitutes = pd.read_csv(r"C:\Users\derek.castleman\Desktop\2025_02_25_20265_Data Analyst Absent Teacher Report_664664824_.csv")
substitutes

# Name the output destination and file name
output = "C:\\Users\\derek.castleman\\Desktop\\Substitutes.xlsx"

In [ ]:
# Selects for absent days related to Teachers and SGIs
substitutes = substitutes[(substitutes['Employee Type'] == 'Small Group Instructor') | 
                         (substitutes['Employee Type'] == 'Teacher')]
substitutes

In [ ]:
# Drop duplicates on date since a teacher can work multiple jobs in one day
substitutes = substitutes.drop_duplicates(subset=['Date', 'Substitute Full Name'])
substitutes

In [ ]:
# GIve a value of 1 for each day worked
substitutes['Days'] = 1
substitutes

In [ ]:
# Selects for Delano sites
delano = substitutes[(substitutes['School Name'] == 'WCPA - Delano Elementary') 
                     | (substitutes['School Name'] == 'WCPA - Delano High School') | 
                    (substitutes['School Name'] == 'WCPA - Delano Middle School') | 
                     (substitutes['School Name'] == 'Primary School-Delano')]
delano

In [ ]:
# Groups by substitute name and sums up the days worked
delano = delano.groupby(['Substitute Full Name']).sum(['Days']).reset_index()
delano

In [ ]:
# Changes day column to say Delano
delano = delano.rename(columns={'Days': 'Delano'})
delano

In [ ]:
# Selects for the Lost Hills schools
losthills = substitutes[(substitutes['School Name'] == 'WCPA - Elementary Lost Hills') 
                     | (substitutes['School Name'] == 'WCPA - Lost Hills Middle School') | 
                    (substitutes['School Name'] == 'WCPA-Lost Hills High School')]
losthills

In [ ]:
# Groups by substitute name and the days worked
losthills = losthills.groupby(['Substitute Full Name']).sum(['Days']).reset_index()
losthills

In [ ]:
# Changes name of Days to Lost Hills
losthills = losthills.rename(columns={'Days': 'Lost Hills'})
losthills

In [ ]:
# Merges Delano dataframe with Lost Hills
final = pd.merge(delano, losthills, how='outer', on='Substitute Full Name')
final

In [ ]:
# Fill NaN values with a 0
final = final.fillna(0)
final

In [ ]:
# Totals columns to get total days worked
final['Total'] = final['Delano'] + final['Lost Hills']
final

In [ ]:
# Puts teachers in alphabetical order
final = final.sort_values(by='Substitute Full Name')
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Substitutes Days Covered", filename = "Substitutes Days Covered"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)

In [ ]:
substitutes_roster

In [ ]:
final

In [ ]:
# Merges Substitute full roster with the days worked one
final = pd.merge(final, substitutes_roster, how='outer', left_on='Substitute Full Name', right_on='Full Name')
final

In [ ]:
# Fills missing Substitute Full Name with the Full Name from roster
final['Substitute Full Name'] = final['Substitute Full Name'].fillna(final['Full Name'])
final

In [ ]:
# Selects final columns
final = final[['Substitute Full Name', 'Delano', 'Lost Hills', 'Total']]
final

In [ ]:
# Fill in missing values with 0
final = final.fillna(0)
final

In [ ]:
# Puts teachers in alphabetical order
final = final.sort_values(by='Substitute Full Name')
final

In [ ]:
# Active substitutes are selected by having more than 0 days of work
active = final[final['Total'] > 0]
active

In [ ]:
# Inactive selects substitutes with 0 days worked
inactive = final[final['Total'] == 0]
inactive

In [ ]:
# Write dataframe to file

writer = pd.ExcelWriter(output)

active.to_excel(writer, sheet_name = 'Active', index=False)
inactive.to_excel(writer, sheet_name='Inactive', index=False)

writer.save()